In [ ]:
import shap
import joblib
import warnings
import pandas as pd
import seaborn as sns
from pdpbox import pdp
from sklearn.svm import SVC
from sklearn import set_config
import matplotlib.pyplot as plt
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from utilities.evaluation import ModelEvaluation
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

set_config(display='diagram')

pd.set_option('float_format', '{:.2f}'.format)
warnings.filterwarnings(action='ignore')

In [ ]:
churn = pd.read_csv('../data/churn-modeling.csv', sep=';')
churn.sample(n=4)

## Validación cruzada

In [ ]:
exited = churn.pop('Exited')

train_data, test_data, train_label, test_label = train_test_split(churn, exited,
                                                                  test_size=0.3,
                                                                  stratify=exited, random_state=42)

print(f'El set de entrenamiento tiene {train_data.shape[0]} observaciones, y {train_data.shape[1]} variables.')
print(f'El set de prueba tiene {test_data.shape[0]} observaciones, y {test_data.shape[1]} variables.')

In [ ]:
numerical_features = ['CreditScore', 'Age', 'Tenure',
                      'Balance', 'NumOfProducts', 'EstimatedSalary']

categorical_features = ['Gender', 'Geography', 'HasCrCard', 'IsActiveMember']

In [ ]:
numeric_preprocessing = Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                                        ('scaler', StandardScaler())])

categoric_preprocessing = Pipeline(steps=[('cat_imputer', SimpleImputer(strategy='most_frequent')),
                                          ('ohe', OneHotEncoder(handle_unknown='ignore', sparse=False))])

preprocessing = ColumnTransformer(transformers=[('numeric', numeric_preprocessing, numerical_features),
                                                ('categorical', categoric_preprocessing, categorical_features)],
                                  remainder='drop')

In [ ]:
lr = Pipeline(steps=[('preprocessing', preprocessing),
                     ('logistic_regression', LogisticRegression(penalty='l1', solver='saga',
                                                                class_weight='balanced', random_state=42, n_jobs=-1))])

dtree = Pipeline(steps=[('preprocessing', preprocessing),
                        ('dtree', DecisionTreeClassifier(max_depth=4,
                                                         class_weight='balanced', random_state=42))])

svm = Pipeline(steps=[('preprocessing', preprocessing),
                      ('svm', SVC(probability=True,
                                  class_weight='balanced', random_state=42))])

In [ ]:
roc = {}
f1_score = {}

roc['logistic_regression'] = cross_val_score(estimator=lr,
                                             X=train_data, y=train_label, scoring='roc_auc', cv=5, n_jobs=-1)
roc['decision_tree'] = cross_val_score(estimator=dtree,
                                       X=train_data, y=train_label, scoring='roc_auc', cv=5, n_jobs=-1)
roc['svm'] = cross_val_score(estimator=svm,
                             X=train_data, y=train_label, scoring='roc_auc', cv=5, n_jobs=-1)

# --------------------------------------------------------------------------------------------------------------

f1_score['logistic_regression'] = cross_val_score(estimator=lr,
                                                  X=train_data, y=train_label, scoring='f1', cv=5, n_jobs=-1)
f1_score['decision_tree'] = cross_val_score(estimator=dtree,
                                            X=train_data, y=train_label, scoring='f1', cv=5, n_jobs=-1)
f1_score['svm'] = cross_val_score(estimator=svm,
                                  X=train_data, y=train_label, scoring='f1', cv=5, n_jobs=-1)


